In [1]:
#loading the packages#

#using Pkg
#Pkg.add("DataFrames")
#Pkg.add("Random")
#Pkg.add("RollingFunctions")
#Pkg.add("XLSX")
#Pkg.add("Distributions") 

using DataFrames
using Random
using RollingFunctions
using XLSX
using Distributions

In [2]:
# importing the dataframe #

#this one has all the values
s7_data_combined = XLSX.readxlsx("C:/Users/15167/OneDrive/Documents/ISB/AML-DT-BNM/raw_data/s7_data_combined.xlsx")
sh = s7_data_combined["Sheet1"]

608×2 XLSX.Worksheet: ["Sheet1"](A1:B608) 

In [3]:
# setting the variables and constants #

n_obs = 60000  # number of steps
n_constant_nodes = ((n_obs) - 5000)  # for the constant nodes (75000 steps)
p = 78  # number of variables
chance_to_flip = 0.01

#78 time-series of noise for 78 variables  #integers not floats
zeta = rand(Uniform(0,1), p, n_obs)

#creates an "empty" time series  #integers not floats
empty_series = zeros(Int, p, n_obs)

# Initial value of the variables were randomly chosen to be 0.0 or 1.0
empty_series[:, 1] = rand(0:1, p)

# Initial value of the variables were randomly chosen to be 0.0 or 1.0 
empty_series[:, n_constant_nodes] = rand(0:1, p)

# Saving the empty time series as the time series (so it can be regenerated later)
time_series = empty_series

# Saving a network_score matrix 
network_scores = zeros(n_obs, 5)

#example mutation_profile to see if function works
mutation_profile = sh["B2"]

#mutation_profile = sh["B1:B20"] #getting all the mutation profiles ONLY
#time_series[1,:] #gives you a row (this would be an individual node in our case) with n.obs

"WT1,FLT3,DNMT3A,DNMT3A"

In [4]:
# score_personal function #

#creating the function for personal score
function score_personal(mutation_profile)
    # Separate the values by commas
    mutation_profile = split(mutation_profile, ',')

    # Remove repeated values
    mutation_profile = unique(mutation_profile)
    
    # Join back together
    join(mutation_profile, ",")

    println(mutation_profile)
    println("preparing to create time series")
    
    for col in (n_constant_nodes + 1) :n_obs
        #simulate the constant nodes for only a couple of steps
        time_series[1, col] = time_series[1, col - 1]  # KITLG = KITLG
        time_series[2, col] = time_series[2, col - 1]  # FLT3LG = FLT3LG
        time_series[4, col] = time_series[4, col - 1]  # KDM5A = KDM5A
        time_series[7, col] = time_series[7, col - 1]  # CBLB = CBLB
        time_series[12, col] = time_series[12, col - 1]  # CDK1 = CDK1
        time_series[13, col] = time_series[13, col - 1]  # STAG2 = STAG2
        time_series[14, col] = time_series[14, col - 1]  # SH2B3 = SH2B3
        time_series[16, col] = time_series[16, col - 1]  # AML1_ETO = AML1_ETO
		time_series[17, col] = time_series[17, col - 1]  # PML_RARalpha = PML_RARalpha
		time_series[37, col] = time_series[37,col - 1] #NUP98_Fusion = NUP98_Fusion
		time_series[38, col] = time_series[38,col - 1] #MLL_Fusion = MLL_Fusion
		time_series[46, col] = time_series[46,col - 1] #NPM1 = NPM1
		time_series[48, col] = time_series[48,col - 1] #IDH2 = IDH2
		time_series[53, col] = time_series[53,col - 1]	#AMPK = AMPK
		time_series[54, col] = time_series[54,col - 1] #ASXL2 = ASXL2
		time_series[58, col] = time_series[58,col - 1]	#DNMT3A = DNMT3A
		time_series[60, col] = time_series[60,col - 1]	#ASXL1 = ASXL1
		time_series[64, col] = time_series[64,col - 1] #PHF6 = PHF6
		time_series[68, col] = time_series[68,col - 1] #SRSF2 = SRSF2
		time_series[70, col] = time_series[70,col - 1]	#BCOR = BCOR
		time_series[74, col] = time_series[74,col - 1]	#U2AF1 = U2AF1
		time_series[75, col] = time_series[75,col - 1]	#XPO1 = XPO1
		time_series[76, col] = time_series[76,col - 1]	#CREBBP = CREBBP
	end
	
	for col in 2:n_obs
        # Simulate each time step based on the previous time step and Boolean functions
        time_series[3, col] = time_series[6, col - 1] & time_series[11, col - 1] & time_series[10, col - 1]  # GRB2 = BCR_ABL | FLT3 | KIT
		time_series[5, col] = abs((time_series[4, col - 1])-1) & time_series[5, col - 1]  # PTEN = !KDM5A & PTEN
		time_series[6, col] = abs((time_series[5, col - 1])-1)  # BCR_ABL = !PTEN
        time_series[8, col] = time_series[3, col - 1]  # CBL = GRB2
        time_series[9, col] = time_series[3, col - 1]  # SOS1 = GRB2  	
		time_series[10, col] = abs((time_series[8, col - 1])-1) & abs((time_series[7, col - 1])-1) & (time_series[16, col - 1] | time_series[1, col - 1])  # KIT = !(CBL | CBLB) & (AML1_ETO | KITLG)
        time_series[11, col] = abs((time_series[8, col - 1])-1) & abs((time_series[7, col - 1])-1) & time_series[2, col - 1]  # FLT3 = !(CBL | CBLB ) & FLT3LG
		time_series[15, col] = abs((time_series[11, col - 1])-1)  # PTPN6 =!FLT3
		time_series[18, col] = time_series[11, col - 1] & time_series[10, col - 1]  # PTPN11 = FLT3 | KIT
		time_series[19, col] = abs((time_series[12, col - 1])-1)  # CUX1 =!CDK1
		time_series[20, col] = time_series[13, col - 1]  # RAD21 = STAG2
		time_series[21, col] = abs((time_series[14, col - 1])-1) & time_series[29, col - 1]  # BCL2L1 = !SH2B3 & STAT5A
		time_series[22, col] = abs((time_series[16,col - 1])-1) & abs((time_series[15,col - 1])-1) & abs((time_series[14,col - 1])-1) & (time_series[6,col - 1] | time_series[10,col - 1]) 	#JAK2 = !(AML1_ETO | PTPN6 | SH2B3) & (BCR_ABL | KIT)
		time_series[23, col] = time_series[17,col - 1]	#CCNA1 = PML_RARalpha
		time_series[24, col] = abs((time_series[16,col - 1])-1) & abs((time_series[11,col - 1])-1) & abs((time_series[17,col - 1])-1) #CEBPA =!(AML1_ETO | FLT3 | PML_RARalpha)
		time_series[25, col] = abs((time_series[19,col - 1])-1)	#PIK3IP1 =!CUX1
		time_series[26, col] = time_series[9,col - 1] & time_series[18,col - 1] #NRAS = SOS1 | PTPN11
		time_series[27, col] = abs((time_series[20,col - 1])-1) & time_series[17,col - 1] #GATA2 = !RAD21 & PML_RARalpha
		time_series[28, col] = abs((time_series[21,col - 1])-1)	#BAX = !BCL2L1
		time_series[29, col] = time_series[22,col - 1] & time_series[6,col - 1]	#STAT5A = JAK2 | BCR_ABL
		time_series[30, col] = abs((time_series[24,col - 1])-1) #SOX4 = !CEBPA
		time_series[31, col] = abs((time_series[25,col - 1])-1) & (time_series[11,col - 1] | time_series[10,col - 1] | time_series[26,col - 1] | time_series[32,col - 1]) #PI3K = !PIK3IP1 & (FLT3 | KIT | NRAS | SPI1)
		time_series[32, col] = abs((time_series[27,col - 1])-1) & time_series[24,col - 1]  #SPI1 = !GATA2 & CEBPA
		time_series[33, col] = time_series[29,col - 1] #PIM = STAT5A
		time_series[34, col] = time_series[30,col - 1] & time_series[11,col - 1] #CTNNB1 =  SOX4 | FLT3 
		time_series[35, col] = time_series[31,col - 1] 	#AKT = PI3K 
		time_series[36, col] = abs((time_series[32,col - 1])-1) & time_series[27,col - 1] #GATA1 = !SPI1 & GATA2
		time_series[39, col] = abs((time_series[35,col - 1])-1) #FOXO = !AKT
		time_series[40, col] = time_series[35,col - 1] #EP300 = AKT
		time_series[41, col] = abs((time_series[35,col - 1])-1) & time_series[26,col - 1] #BRAF = !AKT & NRAS
		time_series[42, col] = time_series[36,col - 1] #CBFB = GATA1
		time_series[43, col] = time_series[37,col - 1] #CDK6 = NUP98_Fusion
		time_series[44, col] = time_series[38,col - 1] #MECOM = MLL_Fusion
		time_series[45, col] = time_series[38,col - 1] #CBFbeta_MYH11 = MLL_Fusion
		time_series[47, col] = time_series[39,col - 1] #IDH1 = FOXO
		time_series[49, col] = time_series[41,col - 1] #ERK1/2 = BRAF
		time_series[50, col] = abs((time_series[38,col - 1])-1) & abs((time_series[45,col - 1])-1) & abs((time_series[20,col - 1])-1) & (time_series[11,col - 1] | time_series[43,col - 1] | time_series[42,col - 1] | time_series[50,col - 1])	#RUNX1 = !(MLL_Fusion | CBFbeta_MYH11 | RAD21) & (FLT3 | CDK6 | CBFB | RUNX1)
		time_series[51, col] = time_series[45,col - 1] #DOT1L = CBFbeta_MYH11
		time_series[52, col] = time_series[46,col - 1] #FBXW7 = NPM1
		time_series[55, col] = time_series[47,col - 1] & time_series[48,col - 1] #OXO2 = IDH1 | IDH2
		time_series[56, col] = abs((time_series[49,col - 1])-1) #ETV6 = !ERK1/2
		time_series[57, col] = time_series[49,col - 1] #AP1 = ERK1/2
		time_series[59, col] = abs((time_series[35,col - 1])-1) #AEZH2 = !AKT
		time_series[61, col] = abs((time_series[52,col - 1])-1) & abs((time_series[50,col - 1])-1) & (time_series[49,col - 1] | time_series[51,col - 1]) #MYC = !(FBXW7 | RUNX1) & (ERK1/2 | DOT1L)
		time_series[62, col] = abs((time_series[53,col - 1])-1) & abs((time_series[35,col - 1])-1) #MTOR = !(AMPK | AKT)
		time_series[63, col] = time_series[55,col - 1] | time_series[53,col - 1] | time_series[54,col - 1]	#TET2 = OXO2 | AMPK | ASXL2
		time_series[65, col] = abs((time_series[58,col - 1])-1) #CCND1 = !DNMT3A
		time_series[66, col] = abs((time_series[59,col - 1])-1) & abs((time_series[58,col - 1])-1) & abs((time_series[60,col - 1])-1) & time_series[51,col - 1]	#HOXA9 = !(AEZH2 | DNMT3A | ASXL1) & DOT1L 
		time_series[67, col] = abs((time_series[58,col - 1])-1) & abs((time_series[61,col - 1])-1) & abs((time_series[16,col - 1])-1) & (time_series[60,col - 1] | time_series[46,col - 1])	#CDKN2A = !( DNMT3A | MYC | AML1_ETO) & (ASXL1 | NPM1)
		time_series[69, col] = time_series[63,col - 1] #WT1 = TET2
		time_series[71, col] = abs((time_series[64,col - 1])-1) #UBTF = !PHF6
		time_series[72, col] = abs((time_series[58,col - 1])-1) & (time_series[66,col - 1] | time_series[51,col - 1]) #MEIS1 =!DNMT3A & (HOXA9 | DOT1L)
		time_series[73, col] = abs((time_series[67,col - 1])-1) & (time_series[68,col - 1] | time_series[77,col - 1] | time_series[35,col - 1]) #MDM2 = !CDKN2A & (SRSF2 | TP53 | AKT)
		time_series[77, col] = abs((time_series[73,col - 1])-1) & abs((time_series[45,col - 1])-1) & abs((time_series[75,col - 1])-1) & abs((time_series[74,col - 1])-1) & time_series[76,col - 1] #TP53 = !(MDM2 | CBFbeta_MYH11 | XPO1 | U2AF1) & (CREBBP)
		time_series[78, col] = abs((time_series[77,col - 1])-1) & (time_series[49,col - 1] | time_series[51,col - 1])	#BCL2 = !TP53 & (ERK1/2 | DOT1L) 
		
		#apply random noise
		noise = rand(p) .< chance_to_flip
		time_series[:, col] .= xor.(time_series[:, col], noise)
		
	end 
	
	#for the oncogenes
	for col in 1:n_obs 
		#knocking in genes
		if any(x -> occursin.("KITLG", x), mutation_profile)
		time_series[1,col] = 1
		end
		
		if any(x -> occursin.("FLT3LG", x), mutation_profile)
		time_series[2,col] = 1
		end
		
		if any(x -> occursin.("GRB2", x), mutation_profile)
		time_series[3,col] = 1
		end
		
		if any(x -> occursin.("KDM5A", x), mutation_profile)
		time_series[4,col] = 1
		end
		
		if any(x -> occursin.("BCR_ABL", x), mutation_profile)
		time_series[6,col] = 1
		end
		
		if any(x -> occursin.("SOS1", x), mutation_profile)
		time_series[9,col] = 1
		end
		
		if any(x -> occursin.("KIT", x), mutation_profile)
		time_series[10,col] = 1
		end
		
		if any(x -> occursin.("FLT3", x), mutation_profile)
		time_series[11,col] = 1
		end
		
		if any(x -> occursin.("AML1_ETO", x), mutation_profile)
		time_series[16,col] = 1
		end
		
		if any(x -> occursin.("PML_RARalpha", x), mutation_profile)
		time_series[17,col] = 1	  #check the lettering!
		end
		
		if any(x -> occursin.("PTPN11", x), mutation_profile)
		time_series[18,col] = 1
		end
		
		if any(x -> occursin.("CUX1", x), mutation_profile)
		time_series[19,col] = 1
		end
		
		if any(x -> occursin.("RAD21", x), mutation_profile)
		time_series[20,col] = 1
		end
		
		if any(x -> occursin.("BCL2L1", x), mutation_profile)
		time_series[21,col] = 1
		end
		
		if any(x -> occursin.("JAK2", x), mutation_profile)
		time_series[22,col] = 1
		end
		
		if any(x -> occursin.("CCNA1", x), mutation_profile)
		time_series[23,col] = 1
		end
		
		if any(x -> occursin.("NRAS", x), mutation_profile)
		time_series[26,col] = 1
		end
		
		if any(x -> occursin.("GATA2", x), mutation_profile)
		time_series[27,col] = 1
		end
		
		if any(x -> occursin.("STAT5A", x), mutation_profile)
		time_series[29,col] = 1
		end
		
		if any(x -> occursin.("SOX4", x), mutation_profile)
		time_series[30,col] = 1
		end
		
		if any(x -> occursin.("PI3K", x), mutation_profile)
		time_series[31,col] = 1
		end
		
		if any(x -> occursin.("PIM", x), mutation_profile)
		time_series[33,col] = 1
		end
		
		if any(x -> occursin.("CTNNB1", x), mutation_profile)
		time_series[34,col] = 1
		end
		
		if any(x -> occursin.("AKT", x), mutation_profile)
		time_series[35,col] = 1
		end
		
		if any(x -> occursin.("GATA1", x), mutation_profile)
		time_series[36,col] = 1
		end
		
		if any(x -> occursin.("MLL_Fusion", x), mutation_profile)
		time_series[38,col] = 1 #check the lettering!
		end
		
		if any(x -> occursin.("BRAF", x), mutation_profile)
		time_series[41,col] = 1
		end
		
		if any(x -> occursin.("CDK6", x), mutation_profile)
		time_series[43,col] = 1
		end
		
		if any(x -> occursin.("MECOM", x), mutation_profile)
		time_series[44,col] = 1
		end
		
		if any(x -> occursin.("NPM1", x), mutation_profile)
		time_series[46,col] = 1
		end
		
		if any(x -> occursin.("IDH1", x), mutation_profile)
		time_series[47,col] = 1
		end
		
		if any(x -> occursin.("IDH2", x), mutation_profile)
		time_series[48,col] = 1
		end
		
		if any(x -> occursin.("ERK", x), mutation_profile)
		time_series[49,col] = 1 #check the lettering!
		end
		
		if any(x -> occursin.("DOT1L", x), mutation_profile)
		time_series[51,col] = 1
		end
		
		if any(x -> occursin.("AMPK", x), mutation_profile)
		time_series[53,col] = 1	#check the value
		end
		
		if any(x -> occursin.("OXO2", x), mutation_profile)
		time_series[55,col] = 1 #check the value
		end
		
		if any(x -> occursin.("AP1", x), mutation_profile)
		time_series[57,col] = 1
		end
		
		if any(x -> occursin.("MYC", x), mutation_profile)
		time_series[61,col] = 1
		end
		
		if any(x -> occursin.("MTOR", x), mutation_profile)
		time_series[62,col] = 1 #check the value
		end
		
		if any(x -> occursin.("CCND1", x), mutation_profile)
		time_series[65,col] = 1
		end
		
		if any(x -> occursin.("HOXA9", x), mutation_profile)
		time_series[66,col] = 1
		end
		
		if any(x -> occursin.("SRSF2", x), mutation_profile)
		time_series[68,col] = 1 #check the value
		end
		
		if any(x -> occursin.("UBTF", x), mutation_profile)
		time_series[71,col] = 1
		end
		
		if any(x -> occursin.("MEIS1", x), mutation_profile)
		time_series[72,col] = 1
		end
		
		if any(x -> occursin.("MDM2", x), mutation_profile)
		time_series[73,col] = 1	#check the value
		end
		
		if any(x -> occursin.("U2AF1", x), mutation_profile)
		time_series[74,col] = 1
		end
		
		if any(x -> occursin.("XPO1", x), mutation_profile)
		time_series[75,col] = 1
		end
		
		if any(x -> occursin.("BCL2", x), mutation_profile)
		time_series[78,col] = 1
		end
	end
	
	#for the TSG
	for col in 1:n_obs 
		#knocking in genes
		if any(x -> occursin.("PTEN", x), mutation_profile)
		time_series[5,col] = 0
		end
		
		if any(x -> occursin.("CBLB", x), mutation_profile)
		time_series[7,col] = 0
		end
		
		if any(x -> occursin.("CBL", x), mutation_profile)
		time_series[8,col] = 0
		end
		
		if any(x -> occursin.("CDK1", x), mutation_profile)
		time_series[12,col] = 0
		end
		
		if any(x -> occursin.("STAG2", x), mutation_profile)
		time_series[13,col] = 0
		end
		
		if any(x -> occursin.("SH2B3", x), mutation_profile)
		time_series[14,col] = 0
		end
		
		if any(x -> occursin.("PTPN6", x), mutation_profile)
		time_series[15,col] = 0
		end
		
		if any(x -> occursin.("CEBPA", x), mutation_profile)
		time_series[24,col] = 0
		end
		
		if any(x -> occursin.("PIK3IP1", x), mutation_profile)
		time_series[25,col] = 0
		end
		
		if any(x -> occursin.("BAX", x), mutation_profile)
		time_series[28,col] = 0
		end
		
		if any(x -> occursin.("SPI1", x), mutation_profile)
		time_series[32,col] = 0
		end
		
		if any(x -> occursin.("NUP98_Fusion", x), mutation_profile)
		time_series[37,col] = 0 #check the lettering!
		end
		
		if any(x -> occursin.("FOXO", x), mutation_profile)
		time_series[39,col] = 0
		end
		
		if any(x -> occursin.("EP300", x), mutation_profile)
		time_series[40,col] = 0
		end
		
		if any(x -> occursin.("CBFB", x), mutation_profile)
		time_series[42,col] = 0
		end
		
		if any(x -> occursin.("CBFbeta_MYH11", x), mutation_profile)
		time_series[45,col] = 0 #check the lettering!
		end
		
		if any(x -> occursin.("RUNX1", x), mutation_profile)
		time_series[50,col] = 0
		end
		
		if any(x -> occursin.("FBXW7", x), mutation_profile)
		time_series[52,col] = 0
		end
		
		if any(x -> occursin.("ASXL2", x), mutation_profile)
		time_series[54,col] = 0
		end
		
		if any(x -> occursin.("ETV6", x), mutation_profile)
		time_series[56,col] = 0
		end
		
		if any(x -> occursin.("DNMT3A", x), mutation_profile)
		time_series[58,col] = 0
		end
		
		if any(x -> occursin.("AEZH2", x), mutation_profile)
		time_series[59,col] = 0
		end
		
		if any(x -> occursin.("ASXL1", x), mutation_profile)
		time_series[60,col] = 0
		end
		
		if any(x -> occursin.("TET2", x), mutation_profile)
		time_series[63,col] = 0
		end
		
		if any(x -> occursin.("PHF6", x), mutation_profile)
		time_series[64,col] = 0
		end
		
		if any(x -> occursin.("CDKN2A", x), mutation_profile)
		time_series[67,col] = 0
		end
		
		if any(x -> occursin.("WT1", x), mutation_profile)
		time_series[69,col] = 0
		end
		
		if any(x -> occursin.("BCOR", x), mutation_profile)
		time_series[70,col] = 0
		end
		
		if any(x -> occursin.("CREBBP", x), mutation_profile)
		time_series[76,col] = 0
		end
		
		if any(x -> occursin.("TP53", x), mutation_profile)
		time_series[77,col] = 0
		end
	end
	
	println("time series is complete")
	
	#computing the scores
	for col in 1:n_obs
		network_scores[col,1] = ((-1 * time_series[67,col]) + (-1 * time_series[69,col]) + (-1 * time_series[70,col]) + time_series[29,col] + time_series[34,col] + time_series[61,col] + time_series[57,col] + time_series[23,col] + time_series[72,col] + time_series[65,col] + time_series[71,col])
		network_scores[col,2]= ((-1 * time_series[30,col]) + ( -1 * time_series[45,col]) + time_series[32,col] + time_series[50,col] + time_series[24,col] + time_series[56,col] + time_series[40,col])
		network_scores[col,3] = ((-1 * time_series[78,col]) + (-1 * time_series[33,col]) + time_series[39,col] + time_series[77,col] + time_series[28,col])
		network_scores[col,4] = (network_scores[col,1]) - ((network_scores[col,2]) + (network_scores[col,3]))
    end 
    
    #computing the average scores from rolling mean
	#rolling window of size n_obs/1000
	window_size = div(n_obs, 2000)
	
	#calculate the rolling average for each variable using the RollingFunctions package
	rolling_avg = rollmean(network_scores[:,4], window_size)
	
	#Compute the final_score
    final_score = mean(rolling_avg[(length(rolling_avg)-2000):length(rolling_avg)], dims=1)
	final_score = final_score[1]
	final_score = round(final_score, sigdigits=3)
    
    proliferation = mean(network_scores[15000:20000,1])
    differentiation = mean(network_scores[15000:20000,2])
    apoptosis = mean(network_scores[15000:20000,3])
    final_score = final_score
    
    score = [proliferation,differentiation,apoptosis,final_score]
    
    return score

end

score_personal (generic function with 1 method)

In [5]:
#testing the patient_id and mutation_profile 
patient_id = sh["A1:A2"]
mutation_profile = sh["B1:B5"] 

#println(patient_id[1])
#println(mutation_profile[1])

5×1 Matrix{Any}:
 "TET2,TET2,NPM1"
 "WT1,FLT3,DNMT3A,DNMT3A"
 "ETV6,FLT3,SRSF2,SRSF2,SETBP1,SETBP1"
 "FLT3,CBLB,CBLB,TET2,TET2,TET2,TET2,NPM1"
 "WT1,FLT3,DNMT3A,DNMT3A"

In [6]:
#result = score_personal(mutation_profile[1])  

In [7]:
# Using the function #
mutation_profile = sh["B1:B608"] #getting all the mutation profiles ONLY
#mutation_profile = sh["B1:B5"] #getting all the mutation profiles ONLY

# Saving a prolif, diff, apoptosis score matrix
scores = zeros(608,4)

for i in 1:(length(mutation_profile))
    
    #using the function
    result = score_personal(mutation_profile[i]) 
    println(result)
    
    #saving the scores
    scores[i,1]=result[1] #proliferation
    scores[i,2]=result[2] #differentiation
    scores[i,3]=result[3] #apoptosis
    scores[i,4]=result[4] #final network score

    time_series = empty_series
end
  

SubString{String}["TET2", "NPM1"]
preparing to create time series
time series is complete
[2.0507898420315938, 4.286742651469706, 0.9860027994401119, -4.99]
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
[1.1815636872625475, 4.060787842431513, 1.0127974405118976, -4.93]
SubString{String}["ETV6", "FLT3", "SRSF2", "SETBP1"]
preparing to create time series
time series is complete
[1.1429714057188562, 2.9928014397120575, 1.0237952409518096, -4.92]
SubString{String}["FLT3", "CBLB", "TET2", "NPM1"]
preparing to create time series
time series is complete
[1.1417716456708658, 3.8184363127374525, 1.016996600679864, -5.94]
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
[1.2505498900219956, 3.693461307738452, 1.0297940411917617, -5.03]
SubString{String}["FLT3", "HOXA1"]
preparing to create time series
time series is complete
[1.1355728854229155, 3.6496700659868027, 1.044391121775645, -5.93]
Sub

[2.874425114977005, 1.8708258348330333, 0.8830233953209358, 4.04]
SubString{String}["TPTE2"]
preparing to create time series
time series is complete
[1.2231553689262147, 1.8410317936412717, 0.9004199160167966, 2.09]
SubString{String}["QSER1", "PTPN11", "FLT3", "SMURF2", "BPTF", "PLD3", "ASXL1", "TET2", "KIAA1109", "C6orf106", "EZH2", "FCN1", "ZRSR2"]
preparing to create time series
time series is complete
[1.2119576084783044, 1.8024395120975805, 0.903619276144771, 2.07]
SubString{String}["KIFC3", "WBP1"]
preparing to create time series
time series is complete
[1.2655468906218756, 1.7606478704259148, 0.9004199160167966, 2.1]
SubString{String}["CSF3R"]
preparing to create time series
time series is complete
[1.2615476904619076, 1.7128574285142972, 0.9000199960007998, 1.1]
SubString{String}["CSMD2", "TOR3A", "PARD3", "PDZD3", "KRTAP4-5", "ALPK2", "ZBTB7A", "HAUS5", "GHR", "KLRG2", "TP53INP1", "KDM6A", "COL4A6"]
preparing to create time series
time series is complete
[1.2501499700059988, 1

time series is complete
[1.5246950609878025, 1.0593881223755248, 0.9860027994401119, 2.98]
SubString{String}["DIRAS3", "F5", "HEATR1", "SLC22A24", "SLAIN1", "SYNGR3", "SRSF2", "SIRT2", "DUXA", "C2orf74", "ABHD16B", "RUNX1", "FOXP1", "PDZRN3", "GPER1", "SLC4A2", "KIAA2026", "BCOR", "PHF6", "BRCC3"]
preparing to create time series
time series is complete
[1.7230553889222155, 0.8140371925614877, 0.994001199760048, 3.09]
SubString{String}["KRT10", "TET2", "CNPY3"]
preparing to create time series
time series is complete
[1.8098380323935213, 1.0299940011997601, 0.997000599880024, 3.03]
SubString{String}["UBE2J2", "HTR1D", "COL24A1", "THBS3", "PGBD5", "CAPN5", "FLT3", "SERPINA11", "PTPRS", "PPP1R13L", "KLK13", "SF3B1", "SULF2", "RUNX1", "UGT2B10", "USP9X"]
preparing to create time series
time series is complete
[1.7900419916016796, 0.7908418316336733, 0.9966006798640272, 3.04]
SubString{String}["PAPSS2", "HMGB1", "CTCF", "APCDD1", "RUVBL2", "SIGLEC10", "DNMT3A", "KLHL30", "ASXL1", "RAC2", "NC

preparing to create time series
time series is complete
[2.6830633873225356, 0.8080383923215357, 1.0483903219356128, 3.9]
SubString{String}["RPS6KA1", "RIT1", "GCSAML", "PLCB3", "DPY19L2", "MRC2", "U2AF1", "BMP2K", "NPR3", "MYL10", "JAK2"]
preparing to create time series
time series is complete
[1.728254349130174, 0.8026394721055788, 1.052989402119576, 2.96]
SubString{String}["KMT2A", "APOBEC1", "TSPAN8", "FLT3", "RNF113B", "SMOC1", "INF2", "IDH2", "GPATCH8", "ZNF560", "SBF1", "NUP210", "NPM1", "ZNF292", "OFD1"]
preparing to create time series
time series is complete
[1.7216556688662268, 0.7820435912817436, 1.045990801839632, 2.93]
SubString{String}["CRY2", "PTPRB", "UNG", "FLT3", "TMC3", "ZNF587", "SPRED2", "FAM194A", "EGR1", "NPM1", "COL28A1"]
preparing to create time series
time series is complete
[1.5936812637472506, 0.7764447110577885, 1.0595880823835233, 2.95]
SubString{String}["SH2D5", "AHDC1", "CLCA2", "CHI3L1", "SHANK2", "ZNF259", "SENP1", "MYO1A", "FLT3", "CCDC74B", "SCN9A", 

time series is complete
[1.7086582683463307, 0.6660667866426715, 1.0641871625674866, -1.62]
SubString{String}["ATAD3B", "ARHGEF10L", "TBC1D10C", "KRAS", "TLN2", "ACAN", "CACNA1G", "MYO5B", "NUMBL", "ARHGAP40", "NBEAL2", "KCTD20", "COL19A1", "IKZF1", "NOBOX", "CACNA1B", "ITGB1BP2"]
preparing to create time series
time series is complete
[1.6840631873625276, 0.6596680663867227, 1.076384723055389, -1.61]
SubString{String}["SLC22A10", "FNDC3A", "DNMT3A", "PLXNB1", "MAP1B", "TLE1"]
preparing to create time series
time series is complete
[2.6178764247150568, 0.6548690261947611, 1.0949810037992402, -0.694]
SubString{String}["TIPRL", "PRKCDBP", "ADRBK1", "EVL", "SRSF2", "RTBDN", "CD37", "IDH1", "SIGLEC1", "RBL1", "TNIK", "ADIPOQ", "DOCK4"]
preparing to create time series
time series is complete
[1.6906618676264746, 0.649870025994801, 1.0945810837832433, -1.69]
SubString{String}["CDKN1B", "PTPN11", "DNMT3A", "PCDH1", "NPM1", "ODF1"]
preparing to create time series
time series is complete
[1.678

SubString{String}["RIT1", "NOC3L", "AP2A2", "WT1", "NAT10", "HMGB1", "MARK3", "TRPM1", "KRT10", "CCDC68", "SLC8A2", "TRPM4", "NLRP13", "AC012313.1", "HOXD13", "PKDREJ", "C6orf58", "TAS2R38", "FAM135B"]
preparing to create time series
time series is complete
[2.743251349730054, 0.5590881823635273, 1.0765846830633874, 1.35]
SubString{String}["MRPL24", "ZRANB1", "MYH7B"]
preparing to create time series
time series is complete
[1.8390321935612877, 0.5502899420115976, 1.0697860427914416, 0.37]
SubString{String}["NRAS", "GJA8", "RIN1", "PZP", "TP53", "ALKBH5", "POLI", "ZNF563", "ANKRD60", "RASA2", "C4orf21", "SLC34A1", "PHYKPL", "PLXNA4", "HUWE1", "L1CAM"]
preparing to create time series
time series is complete
[1.8156368726254748, 0.5482903419316136, 1.0591881623675266, 0.302]
SubString{String}["ZNF195", "WT1", "WNK1", "MON2", "FLT3", "PPL", "AHRR", "MRS2"]
preparing to create time series
time series is complete
[2.6964607078584284, 0.5466906618676265, 1.0713857228554289, 1.37]
SubString{St

time series is complete
[2.828634273145371, 0.6230753849230154, 1.0689862027594481, 0.245]
SubString{String}["ZFPM1", "ANKRD11", "NBEAL1", "ERAP1", "EBF1"]
preparing to create time series
time series is complete
[2.890221955608878, 0.6302739452109578, 1.0715856828634274, 0.267]
SubString{String}["CROCC", "DNAJC1", "CYP2A6", "RUNX1", "U2AF1", "PHF6"]
preparing to create time series
time series is complete
[1.9052189562087583, 0.42451509698060386, 1.0739852029594081, 0.207]
SubString{String}["SMC3", "FOLR4", "ADAMTS15", "FLT3", "ATP2A1", "MYO18A", "DNMT3A", "IDH1", "MYL5"]
preparing to create time series
time series is complete
[1.9270145970805839, 0.6012797440511898, 1.0631873625274946, -0.689]
SubString{String}["KCNJ9", "SMC3", "FOLR4", "ADAMTS15", "FLT3", "NEO1", "ATP2A1", "MYO18A", "DNMT3A", "IDH1", "MYL5", "LGI2", "NEURL1B", "EYS"]
preparing to create time series
time series is complete
[1.9358128374325134, 0.5920815836832634, 1.0741851629674066, -0.751]
SubString{String}["CSF3R", "

SubString{String}["KMT2D", "FLT3", "SF3B1", "IKZF1", "BCOR"]
preparing to create time series
time series is complete
[1.7136572685462907, 0.5574885022995401, 1.086382723455309, 0.787]
SubString{String}["CHD5", "SMC3", "KRAS", "ADNP2", "SCAF1", "LGI2", "TET2"]
preparing to create time series
time series is complete
[1.699260147970406, 0.5744851029794041, 1.0747850429914017, 0.834]
SubString{String}["KIF26A", "NKIRAS2", "ZNF98", "CEBPA", "TMEM214", "MYT1", "MGAT3", "SLC9C1", "THOC2"]
preparing to create time series
time series is complete
[1.693261347730454, 0.34293141371725655, 1.0861827634473105, 0.804]
SubString{String}["TP53"]
preparing to create time series
time series is complete
[1.663867226554689, 0.551489702059588, 1.0625874825034993, 0.797]
SubString{String}["NRAS", "KRAS", "UBE3B", "IHH", "ANO7", "FOXP1"]
preparing to create time series
time series is complete
[1.6506698660267947, 0.5402919416116777, 1.0869826034793042, 0.764]
SubString{String}["CBL", "SF3B1", "TET2"]
preparin

LoadError: InterruptException:

In [12]:
column_names = [:Proliferation, :Differentation, :Apoptosis, :Final]
scores_df = DataFrame(scores, column_names)

Row,Proliferation,Differentation,Apoptosis,Final
,Float64,Float64,Float64,Float64
1,2.05079,4.28674,0.986003,-4.99
2,1.18156,4.06079,1.0128,-4.93
3,1.14297,2.9928,1.0238,-4.92
4,1.14177,3.81844,1.017,-5.94
5,1.25055,3.69346,1.02979,-5.03
6,1.13557,3.64967,1.04439,-5.93
7,1.14917,3.5243,1.04199,-5.94
8,1.38792,3.41892,1.05899,-5.0
9,1.42172,3.33413,1.04819,-5.07


In [9]:
#need to add the patient_id and the mutation_profile to the dataframe

#turn the patient_id and mutation_profile into vectors
patient_id = vec(sh["A1:A608"])
mutation_profile = vec(sh["B1:B608"])

scores_df = hcat(DataFrame(Mutation_Profile = mutation_profile), scores_df)
scores_df = hcat(DataFrame(Patient_Id = patient_id), scores_df)

Row,Patient_Id,Mutation_Profile,Proliferation,Differentation,Apoptosis,Final
,Any,Any,Float64,Float64,Float64,Float64
1,09-00705,"TET2,TET2,NPM1",2.05079,4.28674,0.986003,-4.99
2,10-00136,"WT1,FLT3,DNMT3A,DNMT3A",1.18156,4.06079,1.0128,-4.93
3,10-00172,"ETV6,FLT3,SRSF2,SRSF2,SETBP1,SETBP1",1.14297,2.9928,1.0238,-4.92
4,10-00507,"FLT3,CBLB,CBLB,TET2,TET2,TET2,TET2,NPM1",1.14177,3.81844,1.017,-5.94
5,10-00542,"WT1,FLT3,DNMT3A,DNMT3A",1.25055,3.69346,1.02979,-5.03
6,10-00669,"FLT3,HOXA1",1.13557,3.64967,1.04439,-5.93
7,10-00692,"KRAS,KRAS,SOCS1,DNMT3A,NPM1",1.14917,3.5243,1.04199,-5.94
8,10-00715,"WT1,WT1,WT1,IDH2,IDH2,KIAA1328,KIAA2026,BCOR",1.38792,3.41892,1.05899,-5.0
9,10-00791,"WT1,FLT3,FLT3,IDH2,IDH2,DNMT3A,DNMT3A,NPM1",1.42172,3.33413,1.04819,-5.07


In [10]:
#saving the score matrix as an xlsx file for R 
xlsx_file_path= "C:/Users/15167/OneDrive/Documents/ISB/AML-DT-BNM/raw_data/julia_patient_scores.xlsx"

XLSX.writetable(xlsx_file_path, collect(DataFrames.eachcol(scores_df)), DataFrames.names(scores_df))